This program uses the BERT model trained in BERT Training for 3 Classes.ipynb which consisted of supervised learning to classify molecules into 3 classes. These molecules are saved with their abbreviation and full name (retrieved from the Abrv_Curated file) to help the wet lab team determine the molecule they are labeling. 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle 
bert_regen_scores_path = "/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/bert_regen_score_dict_supervised_3_classes_no_molecules.pkl"
bert_regen_scores_loaded = {} 
with open(bert_regen_scores_path, 'rb') as f:
  bert_regen_scores_loaded = pickle.load(f)

In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
df = pd.DataFrame(bert_regen_scores_loaded).T
df.columns = ['Avg Bert Regen Score', 'Total Sentences Found']
df

,Avg Bert Regen Score,Total Sentences Found
netrin,"[0.0390630542746154, 0.9150156227585214, 0.045...",26
ephrin,"[0.6054071474737659, 0.3233049396221759, 0.071...",19
laminin,"[0.0660904208094585, 0.819856876648574, 0.1140...",539
tenascin,"[0.6472398332861485, 0.22477841361684114, 0.12...",100
cspg,"[0.6661122397805199, 0.22850793509625544, 0.10...",305
...,...,...
plg,"[0.1508750207083267, 0.4574542989762449, 0.391...",16
csl,"[0.3354446310378775, 0.11782911803439834, 0.54...",17
s1p,"[0.9994006156921387, 0.000101857163826935, 0.0...",1
ccc,"[0.4802596971858293, 0.3960788568598218, 0.123...",5


In [ ]:
df['% Inhibitor'] = df.apply(lambda row: row[0][0], axis=1)
df['% Promoter'] = df.apply(lambda row: row[0][1], axis=1)
df['% Neither'] = df.apply(lambda row: row[0][2], axis=1)

In [ ]:
abrv = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/Named_Entity_Recognition/Abrv_Curated.csv", header=None)
abrv.index = abrv[0]
abrv

,0,1,2,3,4,5
0,,,,,,
pi,pi,phosphatidylinositol,NaN,NaN,NaN,NaN
pe,pe,phosphatidylethanolamine,NaN,NaN,NaN,NaN
sm,sm,sphingomyelin,NaN,NaN,NaN,NaN
hrp,hrp,horseradish peroxidase,NaN,NaN,NaN,NaN
ritc,ritc,rhodamine isothiocyanate,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
ma,ma,methacrylate,NaN,NaN,NaN,NaN
moetacl,moetacl,hema-2[2-(methacryloyloxy)ethyl]trimethylammon...,NaN,NaN,NaN,NaN
th-,th-,tyrosine hydroxylase-,NaN,NaN,NaN,NaN


In [ ]:
type(abrv.loc['cspg'][1]) == pd.core.series.Series


True

In [ ]:
['test'] + list(abrv.loc['laminin'][1].values)

AttributeError: ignored

In [ ]:
molecule_to_abrv = {} 
for index, row in df.iterrows():
  try:
    abrvs = abrv.loc[index]
    found = abrv.loc[index][1]
    if type(found) == pd.core.series.Series:
      to_add = [index] + list(abrv.loc[index][1].values)
    else:
      to_add = [index, found]
    molecule_to_abrv[index] = ','.join(to_add)
  except KeyError:
    # if no abbreviation just keep the index 
    print(f"No abrv for {index}")
    molecule_to_abrv[index] = index

molecule_to_abrv

No abrv for netrin
No abrv for ephrin
No abrv for tenascin
No abrv for zymosan
No abrv for oncomodulin
No abrv for y27632
No abrv for tlr2
No abrv for igf1
No abrv for kspg
No abrv for taxol


{'netrin': 'netrin',
 'ephrin': 'ephrin',
 'laminin': 'laminin,laminin ( b',
 'tenascin': 'tenascin',
 'cspg': 'cspg,chondroitin sulfate proteoglycan,chondroitin-6- sulphate proteoglyca n,chon- droitin sulfate proteoglycans',
 'zymosan': 'zymosan',
 'camp': 'camp,clic adenosine monophosphate',
 'pten': 'pten,phosphatase and tensin,phosphatase and tensin homolog,phatase and tensin homolog',
 'cntf': 'cntf,ciliary neurotrophic factor,ciliary neurotrophic factor,ciliary neurotrophic factor,ciliary neurotrophic factor',
 'lif': 'lif,leukemia inhib\xad itory factor,leukemia inhibitory  factor,leukemia inhibitory factor',
 'oncomodulin': 'oncomodulin',
 'stat3': 'stat3,signal transducer and activator of transcription 3,signal transducers and activators of transcription 3,signal transducer and activator of transcription 3',
 'socs3': 'socs3,suppressor of cytokine signaling 3,suppressor of cytokine signaling 3',
 'rhoa': 'rhoa,ras homolog gene family member a,ras homolog gene family member a',

In [ ]:
df['Molecule Name'] = df.apply(lambda row: molecule_to_abrv[row.name], axis=1)
df

,Avg Bert Regen Score,Total Sentences Found,% Inhibitor,% Promoter,% Neither,Molecule Name
netrin,"[0.0390630542746154, 0.9150156227585214, 0.045...",26,0.039063,0.915016,0.045921,netrin
ephrin,"[0.6054071474737659, 0.3233049396221759, 0.071...",19,0.605407,0.323305,0.071288,ephrin
laminin,"[0.0660904208094585, 0.819856876648574, 0.1140...",539,0.066090,0.819857,0.114053,"laminin,laminin ( b"
tenascin,"[0.6472398332861485, 0.22477841361684114, 0.12...",100,0.647240,0.224778,0.127982,tenascin
cspg,"[0.6661122397805199, 0.22850793509625544, 0.10...",305,0.666112,0.228508,0.105380,"cspg,chondroitin sulfate proteoglycan,chondroi..."
...,...,...,...,...,...,...
plg,"[0.1508750207083267, 0.4574542989762449, 0.391...",16,0.150875,0.457454,0.391671,"plg,poly(lactide-co-glycolide),poly(lactide-co..."
csl,"[0.3354446310378775, 0.11782911803439834, 0.54...",17,0.335445,0.117829,0.546726,"csl,cbf1/su(h)/lag1"
s1p,"[0.9994006156921387, 0.000101857163826935, 0.0...",1,0.999401,0.000102,0.000498,"s1p,sphingosine-1-phosphate"
ccc,"[0.4802596971858293, 0.3960788568598218, 0.123...",5,0.480260,0.396079,0.123661,"ccc,cation-chloride-cotransporters"


In [ ]:
df = df[['Molecule Name', '% Inhibitor', '% Promoter', '% Neither', 'Total Sentences Found']]

In [ ]:
df

,Molecule Name,% Inhibitor,% Promoter,% Neither,Total Sentences Found
netrin,netrin,0.039063,0.915016,0.045921,26
ephrin,ephrin,0.605407,0.323305,0.071288,19
laminin,"laminin,laminin ( b",0.066090,0.819857,0.114053,539
tenascin,tenascin,0.647240,0.224778,0.127982,100
cspg,"cspg,chondroitin sulfate proteoglycan,chondroi...",0.666112,0.228508,0.105380,305
...,...,...,...,...,...
plg,"plg,poly(lactide-co-glycolide),poly(lactide-co...",0.150875,0.457454,0.391671,16
csl,"csl,cbf1/su(h)/lag1",0.335445,0.117829,0.546726,17
s1p,"s1p,sphingosine-1-phosphate",0.999401,0.000102,0.000498,1
ccc,"ccc,cation-chloride-cotransporters",0.480260,0.396079,0.123661,5


In [ ]:
df.to_csv("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/bert_regen_score_dict_supervised_3_classes_no_molecules_with_name.csv")